In [9]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import datasets
from torchvision.transforms import Lambda, ToTensor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

dataset = pd.read_csv('/home/pedro/Documentos/Kaggle_DSs/full_data.csv')

y = dataset.iloc[:,-1]
X = dataset.iloc[:,:-1] 
from imblearn import under_sampling, over_sampling

X = pd.concat([X[['age','hypertension','heart_disease','avg_glucose_level','bmi']],pd.get_dummies(data=X[['gender','work_type','ever_married','smoking_status','Residence_type']])],1)
X[['age','avg_glucose_level','bmi']] = RobustScaler().fit_transform(X[['age','avg_glucose_level','bmi']])
X, y = over_sampling.SMOTE(0.5).fit_resample(X=X,y=y)

X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2)

/tmp/ipykernel_433347/2393603853.py:17: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X = pd.concat([X[['age','hypertension','heart_disease','avg_glucose_level','bmi']],pd.get_dummies(data=X[['gender','work_type','ever_married','smoking_status','Residence_type']])],1)
/home/pedro/anaconda3/envs/dc_claas/lib/python3.9/site-packages/imblearn/utils/_validation.py:586: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


In [10]:
X_train = torch.from_numpy(X_train.values.astype(np.float32))
X_test = torch.from_numpy(X_test.values.astype(np.float32))
y_train = torch.from_numpy(np.array(y_train))
y_test = torch.from_numpy(np.array(y_test))

In [11]:
import math
from torch import nn, optim

class tensor_ds(object):
    def __init__(self,tensor:torch.Tensor):
        self.X = tensor[:,:-1]
        self.y = tensor[:,-1]
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self,values):
        return self.X[values], self.y[values]

tensor_train = torch.concat((X_train, y_train.unsqueeze(1)), dim=1)

train_loader = DataLoader(dataset=tensor_ds(tensor_train), batch_size=math.floor(X_train.shape[0]/100), shuffle=True)

class NeuralNetwork(nn.Module):
    def __init__(self,input_size,output_size):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, math.floor(input_size/5)),
            nn.ReLU(),
            nn.Linear(math.floor(input_size/5), math.floor(input_size/10)),
            nn.ReLU(),
            nn.Linear(math.floor(input_size/10), output_size)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        logits = nn.Sigmoid()(logits)
        return logits

model = NeuralNetwork(X_train.shape[1],2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [12]:
import time
for i in range(20):
    for index, (X,y) in enumerate(train_loader):
        
        prediction = model(X)

        loss = criterion(prediction,y.type(torch.long))
        
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        print(f'this is the iteration number {index} - loss = {loss}')
            

this is the iteration number 0 - loss = 0.718582034111023
this is the iteration number 1 - loss = 0.7184553742408752
this is the iteration number 2 - loss = 0.7239313721656799
this is the iteration number 3 - loss = 0.7264008522033691
this is the iteration number 4 - loss = 0.7179931998252869
this is the iteration number 5 - loss = 0.7207323908805847
this is the iteration number 6 - loss = 0.7178671956062317
this is the iteration number 7 - loss = 0.7176427841186523
this is the iteration number 8 - loss = 0.7149031758308411
this is the iteration number 9 - loss = 0.7118441462516785
this is the iteration number 10 - loss = 0.7172747254371643
this is the iteration number 11 - loss = 0.717075526714325
this is the iteration number 12 - loss = 0.7193430066108704
this is the iteration number 13 - loss = 0.7115493416786194
this is the iteration number 14 - loss = 0.7298771739006042
this is the iteration number 15 - loss = 0.7269917130470276
this is the iteration number 16 - loss = 0.719143331

In [13]:
_, indices = torch.max(prediction,1)

In [14]:
indices, y

(tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
 tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0.,
         0., 0., 1., 0., 0.]))

In [15]:
_, indices = torch.max(model(X_test),1)

In [16]:
indices

tensor([0, 0, 1,  ..., 0, 0, 1])

In [17]:
np.sum([(v == y_test[i]).numpy() for i,v  in enumerate(indices)])/len(y_test)

0.8309859154929577

In [18]:
tensor_test = torch.concat((X_test, y_test.unsqueeze(1)), dim=1)

testloader = DataLoader(dataset=tensor_ds(tensor_train), batch_size=math.floor(X_train.shape[0]/100), shuffle=True)

In [28]:
correct_pred = {classname: 0 for classname in ['no_avc','avc']}
total_pred = {classname: 0 for classname in ['no_avc','avc']}

with torch.no_grad():
    for data in testloader:
        symptoms, labels = data
        outputs = model(symptoms)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[['no_avc','avc'][int(label)]] += 1
            total_pred[['no_avc','avc'][int(label)]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: no_avc is 89.7 %
Accuracy for class: avc   is 66.0 %
